# Calculating portfolio returns

In order to build and backtest a portfolio, you need to have the returns of multiple assets in a single object

Our model portfolio consists of the following companies and the weight in our portfolio represented by percentages:

* Bitcoin 20%
* Apple 12%
* Microsoft 15%
* Exxon Mobil 8%
* JP Morgan 9%
* Amazon 10%
* Facebook 5%
* AT&T 16%
* Tesla 5%

Note: The portfolio weights should sum up to 100% in most cases

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
%config InlineBackend.figure_format='retina'

btc = pd.read_csv('BTC-USD.csv', parse_dates=['Date'])
btc = btc.sort_values(by='Date')
btc.set_index('Date', inplace=False)
btc['Date'] = pd.to_datetime(btc['Date']).astype(np.int64)

btc.info()

btc['Returns'] = btc['Adj Close'].pct_change()
btc['Returns'].plot()
plt.show();

In [ ]:
# Define the portfolio weights as a numpy array
#portfolio_weights = np.array([0.20, 0.12, 0.15, 0.08, 0.09, 0.10, 0.05, 0.16, 0.05])

# Calculate the weighted stock returns
#WeightedReturns = btc[:9]['Returns]'.mul(portfolio_weights)
# Calculate the portfolio returns
#btc['Portfolio'] = WeightedReturns.sum()

# Plot cumulative portfolio returns over time
#CumulativeReturns = ((1 + btc['Portfolio']).cumprod()-1)
#CumulativeReturns.plot()
#plt.show();

# THIS CAN'T BE RIGHT

# Calculate the daily returns of the adjusted close price

In [ ]:
btc['Returns'] = btc['Adj Close'].pct_change()

# Check first five rows
print(btc.head())

# Plot the returns column over time
btc['Returns'].plot()
plt.show();

# Return Distributions

In order to analyze the probability of outliers in returns, visualize the historical returns using a histogram

You can use the histogram to show the historical density or frequency of a given range of returns

Note: The outliers on the left tail of the return distribution are what you want to avoid, as they represent large negative daily returns. Outliers on the right hand side of the distribution are normally good events for the security

In [ ]:
# Convert decimal returns into percentage returns
percent_return = btc['Returns']*100

# Drop missing values
returns_plot = percent_return.dropna()

# Plot the returns
plt.hist(returns_plot, bins=75)
plt.xlabel('Percentage return')
plt.ylabel('Frequency')
plt.show();

## Equal weighted portfolios
When comparing portfolios you want to consider performance versus a naive equally-weighted portfolio.
If the portfolio doesn't outperform a simple equally weighted portfolio you might want to consider another strategy, or opt for the naive approach if all else fails.
You can expect equally-weighted portfolios to tend to outperform the market when the largest companies are doing poorly.

In [ ]:
# Number of stocks in your portfolio
numstocks = 8

# Creat array of equal weights across all assests
portfolio_weights_ew = np.repeat(1/numstocks, numstocks)

# Calculate equally weighted portfolio returns
btc['Portfolio_EW'] = btc.iloc[:, 0:numstocks].mul(portfolio_weights_ew, axis=1).sum(axis=1)
print(btc['Portfolio_EW'])

## Market-cap weighted portfolios
When large companies are doing well, market cap weighted portfolios outperform due to the largest weights being assigned to the largest companies.

Note: The BTC data does NOT include market cap so i'll be using arb numbers

In [ ]:
# Create an array of market cap in billions
market_capitalizations = np.array([601.51, 469.25, 349.5, 310.48, 299.77, 356.94, 268.88, 331.57])

# Calculate the market cap weights
mcap_weights = market_capitalizations / sum(market_capitalizations)

# Calculate market cap weighted portfolio returns
btc['Portfolio_Mcap'] = btc.iloc[:, 0:8].mul(mcap_weights, axis=1).sum(axis=1)
print(btc['Portfolio_Mcap'])

## NOTE: The 2 above methods are right in the sense of how to determine them, but the result is NOT correct

## The correlation matrix
The correlation matrix can be used to estimate the linear historical relationship between the returns of multiple assets.
Correlation ranges from -1 to 1. The diagonal of the correlation matrix is always 1. The matrix is symmetric.

In [ ]:
# Calculate the correlation matrix
correlation_matrix = btc.corr()

# Print the matrix
# print(correlation_matrix)

### NOTE: The output is NOT correct. Our BTC dataframe ONLY contains bitcoin information, no other stocks or security information to see a correlation

In [ ]:
import seaborn as sns

sns.heatmap(correlation_matrix,
            annot=True,
            cmap='YlGnBu',
            linewidths=0.3,
            annot_kws={'size': 8})

plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show();

# NOTE: This is the correct method to create a heatmap but our data is only BTC

## The co-variance matrix
The correlation matrix doesn't really tell you anything about the variance of the underlying assets, only the linear relationships between assets.
The co-variance matrix on the other hand contains all of this information, which is useful for portfolio optimization and risk management purposes

In [ ]:
# Calculate the covariance matrix
cov_mat = btc.cov()

# Annualize the co-variance matrix(number of trading days)
cov_mat_annual = cov_mat * 365

# Print annualized covariance matrix
#print(cov_mat_annual)


# Since the variance is the square of vol you didnt have to multiply by the square root of 365

## Portfolio standard deviation

In order to calculate portfolio volatility, you will need the covariance matrix, the portfolio weights, and knowledge of the transpose operation,
The transpose of a numpy array can be calculated using the .T attribute. The np.dot() function is the dot product of two arrays.

NOTE: This is the correct calculation but need to import more data as BTC isn't helping with this.

portfolio_volatility = np.sqrt((np.dot(portfolio_weights_ew.T, np.dot(cov_mat_annual, portfolio_weights_ew))))

## Markowitz Portfolios

* Sharpe ratio: a measure of risk-adjusted return
* MSR: Max Sharpe Ratio portfolio
* GMV: Global Minimum Volatility portfolio

In [ ]:
numstocks = 5
risk_free = 0
btc['Sharpe'] = (btc['Returns']-risk_free)/btc['Volatility']
MSR = btc.sort_values(by=['Sharpe'], ascending=False)
MSR_weights = MSR.iloc[0,0:numstocks]
np.array(MSR_weights)